### This notebook performs an Exploratory Data Analysis on the data from the papers

In [52]:
# Import packages
from pyspark import SparkContext
from pyspark import SQLContext

import spacy
nlp = spacy.load('en_core_web_sm')

import matplotlib as plt

### First we create a Spark Context  and a SQL Context

In [48]:
# checking if a previous Spark session is active
try:
    sc = SparkContext(appName="SDDM", master='local[*]')
    print("Created a SparkContext")
    sqlContext = SQLContext(sc)
    print("Created a SQLContext")
except ValueError:
    print("SparkContext already exists in this scope")

Created a SparkContext
Created a SQLContext


### Then we load the data into a SQLContext Dataframe

In [49]:
df = (sqlContext.read.format('csv').options(header='true').load('/data/s1847503/SDDM/data.csv'))
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            paper_id|               title|        list_authors|            abstract|           full_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|d23c6a066a58dbe5e...|LY6E impairs coro...|['Stephanie Pfaen...|Zoonotic coronavi...|"4 hepatoma cells...|
|618cd102ec5051a05...|Mating strategy i...|['Federica Rosset...|Adenoviruses are ...|across species, w...|
|ac81102667b0d56ed...|Determination of ...|['V&apos;kovski P...|Positive-sense RN...|"8 peptide of onl...|
|35ac99bfeb665ce0a...|ΦX174 Attenuation...|['James T Van Leu...|Natural selection...|"The unequal use ...|
|ad146e228bda4e5a3...|Modelling the epi...|['Marco Claudio T...|                null|Late December 201...|
|d418cdee18b07a0ad...|Title: Metagenomi...|['Langelier', '; ...|Commentary: Lower...|"Lower respirator...|
|1aa3e788fc6b03c14...|Dark Proteome o

In [51]:
print("The dataset consists of %d papers." % df.count())

The dataset consists of 29315 papers.


In [55]:
# Returns a list of words in a given document while removing punctuation, whitespaces and stop words
def lemmas(doc):
    return [word.lemma_ for word in doc if word.is_alpha and not (word.is_punct or word.is_space or word.is_stop)]


# Returns the words in a given document in a bag of words notation 
def bag_of_words(doc):
    bag = {}
    for word in lemmas(doc):
        if word in bag:
            bag[word] += 1
        else:
            bag[word] = 1
    return bag

lemmas = df.select('full_text').rdd.map(nlp)
print("done")
bags = lemmas.map(bag_of_words)

done


In [59]:
lemmas.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 36, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/s1847503/miniconda3/envs/DLNN/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/s1847503/miniconda3/envs/DLNN/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/s1847503/miniconda3/envs/DLNN/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/s1847503/miniconda3/envs/DLNN/lib/python3.7/site-packa

### Don't forget to close the Spark Context when you are done!

In [46]:
sc.stop()

In [18]:
# nlp = spacy.load('en_core_web_sm')

# # Returns a list of words in a given document while removing punctuation, whitespaces and stop words
# def lemmas(doc):
#     return [word.lemma_ for word in doc if word.is_alpha and not (word.is_punct or word.is_space or word.is_stop)]


# # Returns the words in a given document in a bag of words notation 
# def bag_of_words(doc):
#     bag = {}
#     for word in lemmas(doc):
#         if word in bag:
#             bag[word] += 1
#         else:
#             bag[word] = 1
#     return bag

# df['full_text'] = df['full_text'].apply(nlp)
# print("done")
# df['bag_of_words'] = df['full_text'].apply(bag_of_words)
# df.head()

ValueError: [E088] Text of length 1153369 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.